In [ ]:
import numpy as np
import pandas as pd
import os
import cv2

# Define path to the dataset
PATH = "C:/Users/emanu/Universidade/Mestrado/1_Ano/2_Semestre/ACA/Projeto/ACA-Project/AML-Project2/dataset"

# Read the CSV file containing the image paths and labels
train_df = pd.read_csv(os.path.join(PATH, "train_intel-image-classification-csvdata.csv"))
test_df = pd.read_csv(os.path.join(PATH, "test_intel-image-classification-csvdata-kaggle.csv"))

display(train_df)

In [ ]:
y_trainv = train_df['class']
X_trainv = train_df.drop(columns=['class', 'filename', 'number'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_rem, y_train, y_rem = train_test_split(X_trainv, y_trainv, train_size=0.6, random_state=42, stratify=y_trainv)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42, stratify=y_rem)

In [ ]:
def sigmoid(x, der=False):
    if (der==True) : #derivative 
        f = 1/(1+ np.exp(-x))*(1-1/(1+ np.exp(- x)))
    else : # sigmoid
        f = 1/(1+ np.exp(-x))
    return f

# We may employ the Rectifier Linear Unit (ReLU)
def relu(x, der=False):
    if (der == True): # the derivative of the ReLU is the Heaviside Theta
        f = np.heaviside(x, 1)
    else :
        f = np.maximum(x, 0)
    
    return f

In [ ]:
class MultiLayerPerceptron:
    def __init__(self, number_of_inputs, n_targets, n_hidden, number_of_iterations=1000, learning_rate=0.01, output_type='linear'):
        # Netowrk Parameters
        self.number_of_iterations = number_of_iterations
        self.learning_rate = learning_rate
        self.number_of_inputs = number_of_inputs
        self.number_of_outputs = n_targets
        self.n_hidden = n_hidden
        self.output_type = output_type

        #Weight Initialisation
        self.weights1 = (np.random.rand(self.number_of_inputs+1,self.n_hidden)-0.5)*2/np.sqrt(self.number_of_inputs)
        self.weights2 = (np.random.rand(self.n_hidden+1,self.number_of_outputs)-0.5)*2/np.sqrt(self.n_hidden)

    def predict(self, inputs):
        inputs = np.concatenate((inputs,np.ones((inputs.shape[0],1))),axis=1)
        return self.forward_propagation(inputs)
    
    def forward_propagation(self, inputs):
        ## YOUR CODE HERE ##
        # From input to hidden layer
        self.hidden = np.dot(inputs, self.weights1);
        
        # Activation function. You can start with the RELU, and then include other options later. 
        # Do not forget that you need to use the right derivative when updating the weights
        self.hidden = relu(self.hidden)
        
        # Add the bias in the hidden layer
        self.hidden = np.concatenate((self.hidden,-np.ones((np.shape(inputs)[0],1))),axis=1)

        # From hidden layer to output
        outputs = np.dot(self.hidden,self.weights2);
        
        
        # Output with different activation functions
        if self.output_type == 'linear':
            return outputs
        elif self.output_type == 'logistic':
            return sigmoid(outputs)
        elif self.output_type == 'relu':
            return relu(outputs)
        

    def train(self, training_inputs, targets):
        #Add the Bias to the Input Matrix as as a fixed input.
        training_inputs = np.concatenate((training_inputs,np.ones((training_inputs.shape[0],1))),axis=1)
        updatew1 = np.zeros((np.shape(self.weights1)))
        updatew2 = np.zeros((np.shape(self.weights2)))
        
        for n in range(self.number_of_iterations):
            ## YOUR CODE HERE ##
            #Compute the prediction based on the training inputs
            self.predictions = self.forward_propagation(training_inputs)

            #Compute the Error
            error = 0.5 * np.sum((targets - self.predictions)**2)
            if error < 0.1:
                break

            if n % 1000 == 0:
                print("Iteration %d, Loss: %f" % (n, error))

            #Update the weights backwards
            # Different types of output neurons
            if self.output_type == 'linear':
                deltao = (self.predictions-targets) / training_inputs.shape[0]
            elif self.output_type == 'logistic':
                deltao = (self.predictions-targets) * sigmoid(self.predictions, der=True)
            elif self.output_type == 'relu':
                deltao = (self.predictions-targets) * relu(self.predictions, der=True)
            
            deltah = relu(self.hidden, der=True) * (np.dot(deltao, np.transpose(self.weights2)))
            
            updatew2 = self.learning_rate * (np.dot(np.transpose(self.hidden),deltao))
            
            updatew1 = self.learning_rate * (np.dot(np.transpose(training_inputs),deltah[:,:-1]))
            
            self.weights1 = self.weights1 - updatew1
            self.weights2 = self.weights2 - updatew2

In [ ]:
six = np.zeros((X_train.shape[0], 6))
six[np.where(y_train == 0),0] = 1
six[np.where(y_train == 1),1] = 1
six[np.where(y_train == 2),2] = 1
six[np.where(y_train == 3),3] = 1
six[np.where(y_train == 4),4] = 1
six[np.where(y_train == 5),5] = 1

In [ ]:
an = MultiLayerPerceptron(X_train.shape[1], 6, 30, number_of_iterations=10000, output_type='logistic', learning_rate = 0.001)
an.train(X_train, six)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,  mean_squared_error, r2_score

train_predictions = np.argmax(an.predict(X_train), axis=1)
print("Confusion Matrix\n", confusion_matrix(y_train, train_predictions))
print("Accuracy: %.2f %%" % (accuracy_score(y_train, train_predictions) * 100))

In [ ]:
test_predictions = np.argmax(an.predict(X_test), axis=1)
print("Confusion Matrix\n", confusion_matrix(y_test, test_predictions))
print("Accuracy: %.2f %%" % (accuracy_score(y_test, test_predictions) * 100))